<a href="https://colab.research.google.com/github/kevinhincapie/CAPSTONE-PROYECT/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


In [0]:
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


## 1. Download Dataset

In [0]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [0]:
soup = BeautifulSoup(data, 'html.parser')

List in a array


In [0]:
postalCodeList = []
boroughList = []
neighborhoodList = []

LOCATING TABLE AND POSTAL CODE

In [0]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [0]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

LOADING PANDAS DATAFRAME


In [8]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


REMOVING NONASIGNED COLUMNS


In [10]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


Group neighbors

In [11]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae



Neighborhood "Not assigned", will make the value the same as Borough

In [12]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


## 1st question

In [15]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,Central Bay Street
1,M2H\n,North York\n,Hillcrest Village
2,M4B\n,East York\n,Parkview Hill / Woodbine Gardens
3,M1J\n,Scarborough\n,Scarborough Village
4,M4G\n,East York\n,Leaside
5,M4M\n,East Toronto\n,Studio District
6,M1R\n,Scarborough\n,Wexford / Maryvale
7,M9V\n,Etobicoke\n,South Steeles / Silverstone / Humbergate / Jam...
8,M9L\n,North York\n,Humber Summit
9,M5V\n,Downtown Toronto\n,CN Tower / King and Spadina / Railway Lands / ...
